In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
from einops import rearrange,reduce,repeat
from tqdm import trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
print(device)

cuda


# MLP 

In [3]:
class MLPNet(nn.Module):
    def __init__(self, 
                 in_dim, # input dimension
                 out_dim, # output dimension
                 hidden_dim, # hidden dimension
                 num_layers # number of layers
                ):
        
        super(MLPNet, self).__init__()
        
        self.model = [nn.Linear(in_dim, 1024), nn.ReLU()]
        self.model += [nn.Dropout(0.5)]
        self.model += [nn.Linear(1024, 256), nn.ReLU()]
        self.model += [nn.Linear(256, 128), nn.ReLU()]
        #for i in range(num_layers-2):
        #    self.model += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU()]
        
        
        self.model += [nn.Linear(hidden_dim, out_dim)]
        
        self.model = nn.Sequential(*self.model)
        
    def forward(self, inp):
        
        # Flatten the last two dimensions
        inp = inp.reshape(inp.shape[0], -1)
        
        out = self.model(inp)
        
        #bz x outputlength x 2
        return out.reshape(inp.shape[0], -1, 2)

In [4]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(MyLSTM, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        h_t, c_t = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, (h_t, c_t) = self.lstm(x, (h_t, c_t))
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        #out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, (h_t, c_t)
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        h_0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        c_0 =  torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)       
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return h_0, c_0
    



In [5]:
# Autogressive vs. direct mapping
# Batch Norm? 

# Dataset

In [6]:
class ArgoverseDataset(torch.utils.data.Dataset):
    """Dataset class for Argoverse"""
    
    def __init__(self, 
                 data_path,
                 sample_indices):
        super(ArgoverseDataset, self).__init__()
        
        self.data_path = data_path
        self.sample_indices = sample_indices
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        
        # Load one scene
        pkl_path = self.pkl_list[self.sample_indices[idx]]
        with open(pkl_path, 'rb') as f:
            scene = pickle.load(f)
            
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in']
        p_out_raw = scene['p_out'][pred_id]
        v_in_raw = scene['v_in']
        v_out_raw = scene['v_out'][pred_id]
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        # v_out_normalized = v_out_raw / np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
        
        # Convert to float torch tensor
        return torch.from_numpy(inp).float(), torch.from_numpy(p_out_normalized).float()

In [7]:
class RNNdataset(torch.utils.data.Dataset):
    def __init__(self, 
                 data_path,
                 sample_indices):
        self.data_path = data_path
        self.sample_indices = sample_indices
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.sample_indices)
    
    def __getitem__(self,idx):
        # Load one scene
        pkl_path = self.pkl_list[self.sample_indices[idx]]
        with open(pkl_path, 'rb') as f:
            scene = pickle.load(f)
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        mask = np.where(scene['car_mask'] == 1)[0]
    
        
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in']
        p_out_raw = scene['p_out'][pred_id]
        v_in_raw = scene['v_in']
        
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_norm = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_norm = v_in_raw / v_in_norm
        # v_out_normalized = v_out_raw / np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        p_track = p_in_norm[mask].reshape(-1,19*2)
        v_track = v_in_norm[mask].reshape(-1,19*2)
        
        p_agent = p_in_norm[pred_id].reshape(1,-1)
        v_agent = v_in_norm[pred_id].reshape(1,-1)
        new_mask = []
        p_result = []
        v_result = []
        if (len(mask) > 10):
            
            dist = ((p_track - p_agent)**2).sum(axis=-1)
            #print('dist is ',dist.shape)
            new_mask = np.argpartition(dist,10)[:10]
            
            p_result = p_track[new_mask,:]
            v_result = v_track[new_mask,:]
        else:
            p_result = np.zeros((10,38))
            v_result = np.zeros((10,38))
            k = p_track.shape[0]
            #print('slice',k)
            p_result[:k] = p_track
            v_result[:k] = v_track
        
        #print('shape is',p_agent.shape,v_agent.shape,p_result.shape,v_result.shape)
        inp = np.vstack((p_agent,v_agent,p_result,v_result))
        #print('inp shape is',inp.shape)


        p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
        p_out_norm = rearrange(p_out_normalized, 'a b -> (a b)')
        # Convert to float torch tensor
        return torch.from_numpy(inp).float(), torch.from_numpy(p_out_norm).float()



In [8]:
# Try different ways of normalization
# Leverage other features. 

# Hyperparameter

In [9]:
# Grid/Random Search

In [10]:
batch_size = 1024
in_dim = 19*2
out_dim = 30*2
hidden_dim = 128
num_layers = 3
learning_rate = 0.01
decay_rate = 0.95
num_epoch = 1000

# Data Loader

In [11]:
train_path = "./train/train"

# total number of scenes
indices = np.arange(0, 205942)

# train-valid split
np.random.shuffle(indices)
train_indices = indices[:180000]
valid_indices = indices[180000:]

# define datasets
train_set = ArgoverseDataset(train_path, train_indices)
valid_set = ArgoverseDataset(train_path, valid_indices)

train_set = RNNdataset(train_path, train_indices)
valid_set = RNNdataset(train_path, valid_indices)

# create dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=8)

# Model, Loss Function and Optimizer

In [12]:
# RNN, LSTM, 1dCNN, Transformer
# model = MLPNet(in_dim = in_dim, 
#                out_dim = out_dim,
#                hidden_dim = hidden_dim, 
#                num_layers = num_layers).to(device) # move model to gpu 

# Adaptive Moment Estimation computes adaptive learning rates for each parameter. 
# Compute the decaying averages of past and past squared gradients. 
# Instantiate the model with hyperparameters
model = MyLSTM(input_size=in_dim, output_size=out_dim, hidden_dim=60, n_layers=2).to(device)   #maximum number of hidden size is 120



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)  # stepwise learning rate decay
loss_fun = nn.MSELoss()

# Training

In [13]:
def train_epoch(train_loader, model, optimizer, loss_function):
    model.train()
    train_mse = []
    for inp, tgt in train_loader:
        inp = inp.to(device)
        tgt = tgt.to(device)
       
        
        pred,_ = model(inp)
        #print(pred.shape,tgt.shape)
        pred = pred[:,-1, :].squeeze(1)
        loss = loss_function(pred, tgt)
        train_mse.append(loss.item()) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_mse = round(np.sqrt(np.mean(train_mse)),5)
    return train_mse

def eval_epoch(valid_loader, model, loss_function):
    model.eval()
    valid_mse = []
    preds = []
    trues = []
    with torch.no_grad():
        for inp, tgt in valid_loader:
            inp = inp.to(device)
            tgt = tgt.to(device)
            
            loss = 0
            pred,_ = model(inp)
            #print(tgt.shape,pred.shape)
            pred = pred[:,-1, :].squeeze(1)
            loss = loss_function(pred, tgt)
            preds.append(pred.cpu().data.numpy())
            trues.append(tgt.cpu().data.numpy())
            valid_mse.append(loss.item())
            
        preds = np.concatenate(preds, axis = 0)  
        trues = np.concatenate(trues, axis = 0)  
        valid_mse = round(np.sqrt(np.mean(valid_mse)), 5)
    return valid_mse, preds, trues


In [14]:
# Learning Rate Decay
# Dropout
# L1/L2 Regulization

In [15]:
train_rmse = []
valid_rmse = []
min_rmse = 10e8
print(device)
for i in trange(num_epoch):
    start = time.time()

    # model.train() # if you use dropout or batchnorm. 
    train_rmse.append(train_epoch(train_loader, model, optimizer, loss_fun))

    # model.eval()
    val_rmse, val_preds, val_trues = eval_epoch(valid_loader, model, loss_fun)
    valid_rmse.append(val_rmse)

    # save the best model
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model
        print('Get best model')
        # torch.save([best_model, i, get_lr(optimizer)], name + ".pth")

    end = time.time()
    
    # Early Stopping
    if (len(train_rmse) > 100 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break       

    # Learning Rate Decay        
    scheduler.step()
    
    print("Epoch {} | T: {:0.2f} | Train RMSE: {:0.5f} | Valid RMSE: {:0.5f}".format(i + 1, (end-start) / 60, train_rmse[-1], valid_rmse[-1]))

cuda


  0%|          | 1/1000 [00:33<9:25:22, 33.96s/it]

Get best model
Epoch 1 | T: 0.57 | Train RMSE: 0.20877 | Valid RMSE: 0.09874


  0%|          | 2/1000 [01:08<9:31:41, 34.37s/it]

Get best model
Epoch 2 | T: 0.58 | Train RMSE: 0.07061 | Valid RMSE: 0.05747


...

 10%|█         | 104/1000 [51:19<7:19:23, 29.42s/it]

Epoch 104 | T: 0.50 | Train RMSE: 0.02718 | Valid RMSE: 0.02812


 10%|█         | 105/1000 [51:48<7:17:48, 29.35s/it]

Epoch 105 | T: 0.49 | Train RMSE: 0.02716 | Valid RMSE: 0.02812


 10%|█         | 105/1000 [52:17<7:25:44, 29.88s/it]


# Evaluation and Submission

In [16]:
test_path = "./val_in/val_in/"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
for idx in range(len(test_pkl_list)):
    with open(test_pkl_list[idx], 'rb') as f:
        test_sample = pickle.load(f)
        pred_id = np.where(test_sample["track_id"] == test_sample['agent_id'])[0][0]
        mask = np.where(test_sample['car_mask'] == 1)[0]
        # input: p_in & v_in; output: p_out
        
        p_in_raw = test_sample['p_in']
        #p_out_raw = test_sample['p_out'][pred_id]
        v_in_raw = test_sample['v_in']
        lane_scene = test_sample['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_norm = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_norm = v_in_raw / v_in_norm
        # v_out_normalized = v_out_raw / np.linalg.norm(v_out_raw, axis=1, keepdims=True)
        p_track = p_in_norm[mask].reshape(-1,19*2)
        v_track = v_in_norm[mask].reshape(-1,19*2)
        
        p_agent = p_in_norm[pred_id].reshape(1,-1)
        v_agent = v_in_norm[pred_id].reshape(1,-1)
        new_mask = []
        p_result = []
        v_result = []
        if (len(mask) > 10):
            
            dist = ((p_track - p_agent)**2).sum(axis=-1)
            #print('dist is ',dist.shape)
            new_mask = np.argpartition(dist,10)[:10]
            
            p_result = p_track[new_mask,:]
            v_result = v_track[new_mask,:]
        else:
            p_result = np.zeros((10,38))
            v_result = np.zeros((10,38))
            k = p_track.shape[0]
            #print('slice',k)
            p_result[:k] = p_track
            v_result[:k] = v_track
        
        #print('shape is',p_agent.shape,v_agent.shape,p_result.shape,v_result.shape)
        inp = np.vstack((p_agent,v_agent,p_result,v_result)).reshape(1,22,38)
        inp = torch.from_numpy(inp).to(device,dtype=torch.float)
        preds,_ = model(inp)
        preds = preds.squeeze(0)
        pred = preds[-1, :].cpu().data.numpy()
        pred = rearrange(pred, "(b c) -> b c", c =2 )
        # De-Normalization ! 
        pred = pred * (max_vecs[:2] - min_vecs[:2]) +  min_vecs[:2]
        test_preds.append(pred)

# Generate Submission File

In [17]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

In [18]:
# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1).astype(int)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df.to_csv('test_submission.csv', index=None)